# 🧠 Brain Tumor Segmentation using U-Net

**Author:** Jauilson Crisostomo da Silva  
**Institution:** Universidade Federal de Sergipe (UFS)  
**Department:** Departamento de Ciências da Computação  
**Dataset:** BraTS 2019 (Brain Tumor Segmentation Challenge)  
**Date:** May 2023

---

## 📄 Project Context

This project was developed as part of a research proposal submitted to Prof. Dr. Daniel Oliveira Dantas at the Department of Computer Science, UFS, for master's program application. The work implements automatic brain tumor segmentation in MRI images using deep learning techniques.

Subsequently, I enhanced my knowledge by attending classes as a guest student with Prof. Dr. Jugurta Rosa Montalvão Filho in Pattern Recognition and Medical Image Processing, which contributed significantly to my understanding of the field.

---

## 🎯 Objectives

- Implement automated brain tumor segmentation using U-Net architecture
- Process and analyze MRI images from the BraTS 2019 dataset
- Evaluate model performance using Dice coefficient and IoU metrics
- Visualize segmentation results

---

## 🔬 Key Features

- **U-Net architecture** for medical image segmentation
- **Data preprocessing** with normalization and resizing
- **Dice coefficient and IoU** evaluation metrics
- **Visualization** of segmentation results
- **Model checkpointing** and early stopping

---

## 1. Setup and Imports

In [ ]:
# Mount Google Drive (if using Colab)
try:
    from google.colab import drive
    drive.mount('/content/gdrive')
    IN_COLAB = True
    BASE_PATH = '/content/gdrive/My Drive/databrain'
except:
    IN_COLAB = False
    BASE_PATH = './data'  # Local path
    print("Not running in Colab - using local paths")

In [ ]:
# Import required libraries
import os
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from skimage.transform import resize
import warnings
warnings.filterwarnings('ignore')

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {tf.config.list_physical_devices('GPU')}")

## 2. Configuration

In [ ]:
class Config:
    """Configuration parameters for the project"""
    
    # Paths
    DATA_DIR = BASE_PATH
    IMAGES_TRAIN = os.path.join(DATA_DIR, "imagesTr")
    LABELS_TRAIN = os.path.join(DATA_DIR, "labelsTr")
    IMAGES_TEST = os.path.join(DATA_DIR, "imagesTs")
    RESULTS_DIR = "results"
    
    # Model parameters
    IMG_HEIGHT = 128
    IMG_WIDTH = 128
    IMG_CHANNELS = 1  # Grayscale MRI
    NUM_CLASSES = 1   # Binary segmentation (background vs tumor)
    
    # Training parameters
    BATCH_SIZE = 8
    EPOCHS = 50
    LEARNING_RATE = 1e-4
    VALIDATION_SPLIT = 0.2
    
    # Data range
    START_INDEX = 1
    END_INDEX = 484
    
# Create results directory
config = Config()
os.makedirs(config.RESULTS_DIR, exist_ok=True)

print("Configuration loaded successfully!")
print(f"Data directory: {config.DATA_DIR}")
print(f"Image size: {config.IMG_HEIGHT}x{config.IMG_WIDTH}")
print(f"Batch size: {config.BATCH_SIZE}")
print(f"Epochs: {config.EPOCHS}")

## 3. Data Loading and Preprocessing

In [ ]:
def load_nifti_file(filepath):
    """Load a NIfTI file and return the image data"""
    try:
        img = nib.load(filepath)
        data = img.get_fdata()
        return data
    except Exception as e:
        print(f"Error loading {filepath}: {e}")
        return None

def normalize_image(image):
    """Normalize image to [0, 1] range"""
    img_min = np.min(image)
    img_max = np.max(image)
    if img_max - img_min > 0:
        normalized = (image - img_min) / (img_max - img_min)
    else:
        normalized = image
    return normalized

def preprocess_volume(volume, target_size=(128, 128)):
    """
    Preprocess a 3D MRI volume:
    - Normalize intensity
    - Extract middle slice
    - Resize to target size
    """
    # Normalize
    volume_normalized = normalize_image(volume)
    
    # Extract middle slice from 3D volume
    if len(volume_normalized.shape) == 4:
        # If 4D (e.g., multi-modal), take first modality
        volume_normalized = volume_normalized[:, :, :, 0]
    
    if len(volume_normalized.shape) == 3:
        middle_slice = volume_normalized.shape[2] // 2
        slice_2d = volume_normalized[:, :, middle_slice]
    else:
        slice_2d = volume_normalized
    
    # Resize to target size
    resized = resize(slice_2d, target_size, preserve_range=True, anti_aliasing=True)
    
    # Add channel dimension
    resized = np.expand_dims(resized, axis=-1)
    
    return resized.astype(np.float32)

print("Preprocessing functions defined successfully!")

In [ ]:
def load_dataset(config, max_samples=None):
    """
    Load and preprocess the BraTS dataset
    
    Args:
        config: Configuration object
        max_samples: Maximum number of samples to load (for testing)
    
    Returns:
        images: Preprocessed images array
        labels: Preprocessed labels array
    """
    images = []
    labels = []
    
    # Determine range
    end_index = min(config.END_INDEX, max_samples) if max_samples else config.END_INDEX
    
    print(f"Loading dataset from index {config.START_INDEX} to {end_index}...")
    print("This may take a few minutes...\n")
    
    loaded_count = 0
    
    for i in range(config.START_INDEX, end_index + 1):
        # Generate filename
        file_index = str(i).zfill(3)
        image_path = os.path.join(config.IMAGES_TRAIN, f"BRATS_{file_index}.nii.gz")
        label_path = os.path.join(config.LABELS_TRAIN, f"BRATS_{file_index}.nii.gz")
        
        # Check if files exist
        if not os.path.exists(image_path) or not os.path.exists(label_path):
            continue
        
        # Load volumes
        image_volume = load_nifti_file(image_path)
        label_volume = load_nifti_file(label_path)
        
        if image_volume is None or label_volume is None:
            continue
        
        # Preprocess
        try:
            image_processed = preprocess_volume(
                image_volume,
                target_size=(config.IMG_HEIGHT, config.IMG_WIDTH)
            )
            label_processed = preprocess_volume(
                label_volume,
                target_size=(config.IMG_HEIGHT, config.IMG_WIDTH)
            )
            
            # Binarize label (tumor vs background)
            label_processed = (label_processed > 0).astype(np.float32)
            
            images.append(image_processed)
            labels.append(label_processed)
            
            loaded_count += 1
            
            if (loaded_count % 20) == 0:
                print(f"✓ Processed {loaded_count} images...")
                
        except Exception as e:
            print(f"✗ Error processing BRATS_{file_index}: {e}")
            continue
    
    print(f"\n✅ Successfully loaded {len(images)} images")
    
    return np.array(images), np.array(labels)

print("Dataset loading function defined successfully!")

In [ ]:
# Load dataset
# Note: Start with a small subset (e.g., 50) for testing
# Remove max_samples parameter to load full dataset

print("Starting dataset loading...\n")
X, y = load_dataset(config, max_samples=50)  # Use 50 samples for quick testing

print(f"\nDataset loaded successfully!")
print(f"Images shape: {X.shape}")
print(f"Labels shape: {y.shape}")
print(f"Images dtype: {X.dtype}")
print(f"Labels dtype: {y.dtype}")
print(f"Images range: [{X.min():.3f}, {X.max():.3f}]")
print(f"Labels unique values: {np.unique(y)}")

## 4. Data Exploration and Visualization

In [ ]:
# Visualize sample images
def visualize_samples(images, labels, num_samples=5):
    """Visualize sample images and their labels"""
    fig, axes = plt.subplots(num_samples, 2, figsize=(10, 3*num_samples))
    
    for i in range(num_samples):
        # Original MRI
        axes[i, 0].imshow(images[i, :, :, 0], cmap='gray')
        axes[i, 0].set_title(f'MRI Scan #{i+1}')
        axes[i, 0].axis('off')
        
        # Tumor mask
        axes[i, 1].imshow(labels[i, :, :, 0], cmap='hot')
        axes[i, 1].set_title(f'Tumor Mask #{i+1}')
        axes[i, 1].axis('off')
    
    plt.tight_layout()
    plt.savefig(os.path.join(config.RESULTS_DIR, 'sample_data.png'), dpi=300, bbox_inches='tight')
    plt.show()

print("Visualizing sample data...\n")
visualize_samples(X, y, num_samples=5)

## 5. Train-Validation Split

In [ ]:
# Split data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    X, y, 
    test_size=config.VALIDATION_SPLIT, 
    random_state=42
)

print("Data split completed!")
print(f"Training samples: {len(X_train)}")
print(f"Validation samples: {len(X_val)}")
print(f"\nTraining set shape: {X_train.shape}")
print(f"Validation set shape: {X_val.shape}")

## 6. U-Net Model Architecture

In [ ]:
def conv_block(inputs, num_filters):
    """Convolutional block with two conv layers"""
    x = layers.Conv2D(num_filters, 3, activation='relu', padding='same',
                      kernel_initializer='he_normal')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(num_filters, 3, activation='relu', padding='same',
                      kernel_initializer='he_normal')(x)
    x = layers.BatchNormalization()(x)
    return x

def encoder_block(inputs, num_filters):
    """Encoder block with conv block and max pooling"""
    x = conv_block(inputs, num_filters)
    p = layers.MaxPooling2D((2, 2))(x)
    return x, p

def decoder_block(inputs, skip_features, num_filters):
    """Decoder block with upsampling and concatenation"""
    x = layers.Conv2DTranspose(num_filters, (2, 2), strides=2, padding='same')(inputs)
    x = layers.concatenate([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_unet(input_shape, num_classes=1):
    """
    Build U-Net architecture for image segmentation
    
    Args:
        input_shape: Tuple (height, width, channels)
        num_classes: Number of output classes
    
    Returns:
        Keras model
    """
    inputs = keras.Input(shape=input_shape)
    
    # Encoder (Contracting Path)
    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)
    
    # Bottleneck
    b1 = conv_block(p4, 1024)
    
    # Decoder (Expanding Path)
    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)
    
    # Output layer
    if num_classes == 1:
        outputs = layers.Conv2D(1, 1, activation='sigmoid', padding='same')(d4)
    else:
        outputs = layers.Conv2D(num_classes, 1, activation='softmax', padding='same')(d4)
    
    model = keras.Model(inputs=inputs, outputs=outputs, name='U-Net')
    
    return model

print("U-Net architecture defined successfully!")

## 7. Evaluation Metrics

In [ ]:
def dice_coefficient(y_true, y_pred, smooth=1e-6):
    """
    Dice coefficient metric for segmentation evaluation
    
    Args:
        y_true: Ground truth masks
        y_pred: Predicted masks
        smooth: Smoothing factor to avoid division by zero
    
    Returns:
        Dice coefficient value
    """
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (
        tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + smooth
    )

def dice_loss(y_true, y_pred):
    """Dice loss function"""
    return 1 - dice_coefficient(y_true, y_pred)

def iou_coefficient(y_true, y_pred, smooth=1e-6):
    """
    Intersection over Union (IoU) metric
    
    Args:
        y_true: Ground truth masks
        y_pred: Predicted masks
        smooth: Smoothing factor
    
    Returns:
        IoU value
    """
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    union = tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) - intersection
    return (intersection + smooth) / (union + smooth)

print("Evaluation metrics defined successfully!")

## 8. Model Compilation and Training

In [ ]:
# Build model
input_shape = (config.IMG_HEIGHT, config.IMG_WIDTH, config.IMG_CHANNELS)
model = build_unet(input_shape, num_classes=config.NUM_CLASSES)

# Compile model
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=config.LEARNING_RATE),
    loss='binary_crossentropy',  # or dice_loss
    metrics=['accuracy', dice_coefficient, iou_coefficient]
)

# Print model summary
print("="*80)
print("MODEL ARCHITECTURE")
print("="*80)
model.summary()

print(f"\nTotal parameters: {model.count_params():,}")

In [ ]:
# Define callbacks
callbacks = [
    ModelCheckpoint(
        os.path.join(config.RESULTS_DIR, 'best_unet_model.h5'),
        monitor='val_dice_coefficient',
        mode='max',
        save_best_only=True,
        verbose=1
    ),
    EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-7,
        verbose=1
    )
]

print("Callbacks configured successfully!")

In [ ]:
# Train model
print("\n" + "="*80)
print("TRAINING MODEL")
print("="*80 + "\n")

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    batch_size=config.BATCH_SIZE,
    epochs=config.EPOCHS,
    callbacks=callbacks,
    verbose=1
)

print("\n" + "="*80)
print("TRAINING COMPLETED!")
print("="*80)

## 9. Results Visualization

In [ ]:
def plot_training_history(history):
    """Plot training and validation metrics"""
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Loss
    axes[0, 0].plot(history.history['loss'], label='Train Loss', linewidth=2)
    axes[0, 0].plot(history.history['val_loss'], label='Val Loss', linewidth=2)
    axes[0, 0].set_title('Model Loss', fontsize=14, fontweight='bold')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # Accuracy
    axes[0, 1].plot(history.history['accuracy'], label='Train Accuracy', linewidth=2)
    axes[0, 1].plot(history.history['val_accuracy'], label='Val Accuracy', linewidth=2)
    axes[0, 1].set_title('Model Accuracy', fontsize=14, fontweight='bold')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Accuracy')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # Dice Coefficient
    axes[1, 0].plot(history.history['dice_coefficient'], label='Train Dice', linewidth=2)
    axes[1, 0].plot(history.history['val_dice_coefficient'], label='Val Dice', linewidth=2)
    axes[1, 0].set_title('Dice Coefficient', fontsize=14, fontweight='bold')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('Dice')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # IoU
    axes[1, 1].plot(history.history['iou_coefficient'], label='Train IoU', linewidth=2)
    axes[1, 1].plot(history.history['val_iou_coefficient'], label='Val IoU', linewidth=2)
    axes[1, 1].set_title('IoU Coefficient', fontsize=14, fontweight='bold')
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('IoU')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(os.path.join(config.RESULTS_DIR, 'training_history.png'), 
                dpi=300, bbox_inches='tight')
    plt.show()

# Plot training history
plot_training_history(history)

In [ ]:
def visualize_predictions(model, X_test, y_test, num_samples=5):
    """Visualize model predictions"""
    predictions = model.predict(X_test[:num_samples])
    
    fig, axes = plt.subplots(num_samples, 3, figsize=(12, 4*num_samples))
    
    for i in range(num_samples):
        # Original image
        axes[i, 0].imshow(X_test[i, :, :, 0], cmap='gray')
        axes[i, 0].set_title('Original MRI', fontweight='bold')
        axes[i, 0].axis('off')
        
        # Ground truth
        axes[i, 1].imshow(y_test[i, :, :, 0], cmap='hot')
        axes[i, 1].set_title('Ground Truth', fontweight='bold')
        axes[i, 1].axis('off')
        
        # Prediction
        axes[i, 2].imshow(predictions[i, :, :, 0], cmap='hot')
        axes[i, 2].set_title('Prediction', fontweight='bold')
        axes[i, 2].axis('off')
    
    plt.tight_layout()
    plt.savefig(os.path.join(config.RESULTS_DIR, 'segmentation_results.png'), 
                dpi=300, bbox_inches='tight')
    plt.show()

# Visualize predictions
print("Visualizing predictions...\n")
visualize_predictions(model, X_val, y_val, num_samples=5)

## 10. Model Evaluation

In [ ]:
# Evaluate model on validation set
print("\n" + "="*80)
print("MODEL EVALUATION")
print("="*80 + "\n")

results = model.evaluate(X_val, y_val, verbose=1)

print(f"\n{'='*80}")
print("FINAL RESULTS")
print(f"{'='*80}")
print(f"Validation Loss:            {results[0]:.4f}")
print(f"Validation Accuracy:        {results[1]:.4f}")
print(f"Validation Dice Coefficient: {results[2]:.4f}")
print(f"Validation IoU:             {results[3]:.4f}")
print(f"{'='*80}")

## 11. Save Model

In [ ]:
# Save final model
model_path = os.path.join(config.RESULTS_DIR, 'brain_tumor_segmentation_final.h5')
model.save(model_path)
print(f"✅ Model saved successfully to: {model_path}")

# Save model architecture as JSON
model_json = model.to_json()
with open(os.path.join(config.RESULTS_DIR, 'model_architecture.json'), 'w') as json_file:
    json_file.write(model_json)
print(f"✅ Model architecture saved successfully")

## 12. Summary and Conclusions

### Project Outcomes

This project successfully implemented an automated brain tumor segmentation system using U-Net architecture on the BraTS 2019 dataset.

### Key Achievements

1. **Data Processing:** Successfully preprocessed 484 3D MRI volumes from the BraTS dataset
2. **Model Architecture:** Implemented U-Net with encoder-decoder structure optimized for medical image segmentation
3. **Evaluation Metrics:** Achieved measurable results using Dice coefficient and IoU metrics
4. **Visualization:** Created comprehensive visualizations of training progress and segmentation results

### Technical Skills Demonstrated

- **Deep Learning:** TensorFlow/Keras, CNN architectures, U-Net
- **Medical Image Processing:** NIfTI format handling, 3D volume processing
- **Data Science:** Preprocessing, normalization, train-test splitting
- **Python Programming:** NumPy, scikit-image, Matplotlib
- **Evaluation:** Dice coefficient, IoU, accuracy metrics

### Future Improvements

- Implement 3D U-Net for full volume segmentation
- Add data augmentation techniques
- Experiment with attention mechanisms
- Deploy model as web application
- Test on additional datasets for generalization

---

**Author:** Jauilson Crisostomo da Silva  
**Contact:** jauilson@gmail.com  
**LinkedIn:** [linkedin.com/in/jauilson](https://linkedin.com/in/jauilson)  
**Lattes:** [lattes.cnpq.br/4402347929712204](http://lattes.cnpq.br/4402347929712204)

---